In [ ]:
!pip3 install datasets transformers

In [ ]:
import torch
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer

In [ ]:
torch.cuda.is_available()

True

In [ ]:
from datasets import load_dataset
imdb = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [37]:
train_length = len(imdb["train"])

In [38]:
test_length = len(imdb["test"])

In [39]:
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(train_length))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(test_length))])

In [40]:
small_train_dataset.to_pandas().head(10)

,text,label
0,There is no relation at all between Fortier an...,1
1,This movie is a great. The plot is very true t...,1
2,"George P. Cosmatos' ""Rambo: First Blood Part I...",0
3,In the process of trying to establish the audi...,1
4,"Yeh, I know -- you're quivering with excitemen...",0
5,While this movie's style isn't as understated ...,1
6,I give this movie 7 out of 10 because the vill...,1
7,"really awful... lead actor did OK... the film,...",0
8,Good grief I can't even begin to describe how ...,0
9,Home Room deals with a Columbine-like high-sch...,1


DistilBERT is a small, fast, cheap and light Transformer model based on the BERT architecture. 

Knowledge distillation is performed during the pre-training phase to reduce the size of a BERT model by 40%.

https://www.techtarget.com/searchenterpriseai/definition/BERT-language-model

distilbert-base-uncased

https://huggingface.co/distilbert-base-uncased



In [41]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapsh

In [42]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)
 
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

**To speed up training, let's use a data_collator to convert your training samples to PyTorch tensors and concatenate them with the correct amount of padding**

In [43]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

**Define a Model**

In [46]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when in

**Configuration For Training The Model**

In [47]:
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}


training_args = TrainingArguments(
   output_dir="/content/model/",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=False,
)
 

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [48]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3126
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster 

Step,Training Loss
500,0.327500
1000,0.244300
1500,0.219600
2000,0.156800
2500,0.145600
3000,0.149400


Saving model checkpoint to /content/model/checkpoint-1563
Configuration saved in /content/model/checkpoint-1563/config.json
Model weights saved in /content/model/checkpoint-1563/pytorch_model.bin
tokenizer config file saved in /content/model/checkpoint-1563/tokenizer_config.json
Special tokens file saved in /content/model/checkpoint-1563/special_tokens_map.json
Saving model checkpoint to /content/model/checkpoint-3126
Configuration saved in /content/model/checkpoint-3126/config.json
Model weights saved in /content/model/checkpoint-3126/pytorch_model.bin
tokenizer config file saved in /content/model/checkpoint-3126/tokenizer_config.json
Special tokens file saved in /content/model/checkpoint-3126/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3126, training_loss=0.20383423654528207, metrics={'train_runtime': 2321.6876, 'train_samples_per_second': 21.536, 'train_steps_per_second': 1.346, 'total_flos': 6559874582978592.0, 'train_loss': 0.20383423654528207, 'epoch': 2.0})

In [56]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 16


{'eval_loss': 0.22768324613571167,
 'eval_accuracy': 0.93092,
 'eval_f1': 0.9310386135846345,
 'eval_runtime': 425.287,
 'eval_samples_per_second': 58.784,
 'eval_steps_per_second': 3.675,
 'epoch': 2.0}

In [50]:
trainer.save_model("/content/trained_model/")

Saving model checkpoint to /content/trained_model/
Configuration saved in /content/trained_model/config.json
Model weights saved in /content/trained_model/pytorch_model.bin
tokenizer config file saved in /content/trained_model/tokenizer_config.json
Special tokens file saved in /content/trained_model/special_tokens_map.json


In [51]:
!zip -r trained.zip /content/trained_model/

  adding: content/trained_model/ (stored 0%)
  adding: content/trained_model/vocab.txt (deflated 53%)
  adding: content/trained_model/special_tokens_map.json (deflated 42%)
  adding: content/trained_model/pytorch_model.bin (deflated 8%)
  adding: content/trained_model/tokenizer_config.json (deflated 41%)
  adding: content/trained_model/tokenizer.json (deflated 71%)
  adding: content/trained_model/config.json (deflated 46%)
  adding: content/trained_model/training_args.bin (deflated 48%)


In [52]:
!cp trained.zip /content/drive/MyDrive/imdb_full/

In [53]:
from transformers import TextClassificationPipeline

model = AutoModelForSequenceClassification.from_pretrained("/content/trained_model/", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

loading configuration file /content/trained_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/trained_model/",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading weights file /content/trained_model/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at /content/trained_model/.
If your t

In [54]:
pipe("I love this movie!")
# LABEL_0 = not happy
# LABEL 1 = happy

[[{'label': 'LABEL_0', 'score': 0.004990999586880207},
  {'label': 'LABEL_1', 'score': 0.9950089454650879}]]

In [55]:
pipe("It's just looks to be a copy of the old movie i used to love")
# LABEL_0 = not happy
# LABEL 1 = happy

[[{'label': 'LABEL_0', 'score': 0.5127687454223633},
  {'label': 'LABEL_1', 'score': 0.4872312545776367}]]